In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

import matplotlib.pyplot as plt
import fitsio
import glob

import numpy as np
import healpy as hp
import xlens
import treecorr
import astropy.io.ascii as pyascii
import numpy.lib.recfunctions as rfn
from lsst.geom import degrees
import scipy

from lsst.daf.butler import Butler
from lsst.skymap.ringsSkyMap import RingsSkyMap, RingsSkyMapConfig

In [2]:
if False:
    # Set up the configuration
    config = RingsSkyMapConfig()
    config.numRings = 120
    config.projection = "TAN"
    config.tractOverlap = 1.0 / 60  # degrees
    config.pixelScale = 0.168  # arcsec/pixel
    skymap = RingsSkyMap(config)
    
    # example list of tracts
    tract_list = fitsio.read("/gpfs02/work/xiangchong.li/work/hsc_data/s23b/tracts_id.fits")
    ra = np.zeros(len(tract_list))
    dec = np.zeros(len(tract_list))
    for i, tractId in enumerate(tract_list):
        # get the TractInfo for this tract
        tractInfo = skymap[tractId]
        # pull out its WCS and bounding box
        wcs  = tractInfo.getWcs()
        bbox = tractInfo.getBBox()
        ra_deg, dec_deg = wcs.pixelToSky(
            bbox.getCenter()
        ).getPosition(degrees)
        ra[i] = ra_deg
        dec[i] = dec_deg
    dd = np.zeros(len(tract_list), dtype=[("tract", "i4"), ("ra", "f8"), ("dec", "f8")])
    dd["tract"] = tract_list
    dd["ra"] = ra
    dd["dec"] = dec
    fitsio.write("/gpfs02/work/xiangchong.li/work/hsc_data/s23b/tracts.fits", dd)
else:
    dd = fitsio.read("/gpfs02/work/xiangchong.li/work/hsc_data/s23b/tracts.fits")

In [3]:
# Set up the configuration
config = RingsSkyMapConfig()
config.numRings = 120
config.projection = "TAN"
config.tractOverlap = 1.0 / 60  # degrees
config.pixelScale = 0.168  # arcsec/pixel
skymap = RingsSkyMap(config)

# example list of tracts
tract_list = fitsio.read("/gpfs02/work/xiangchong.li/work/hsc_data/s23b/tracts_id.fits")
ra = np.zeros(len(tract_list))
dec = np.zeros(len(tract_list))
# get the TractInfo for this tract
tractInfo = skymap[0]
bbox = tractInfo.getBBox()

In [4]:
if False:
    frames = pyascii.read("/gpfs02/work/xiangchong.li/work/hsc_data/s23b/frames.csv")
    sel = ((frames["filter"] == "HSC-I") | (frames["filter"] == "HSC-I2")) & (np.abs(frames["ccd"] -49.5) < 0.6)
    frames = frames[sel]
    NSIDE = 1024
    visits = frames['visit']
    ccds   = frames['ccd']
    
    # 1) find all visits that have at least one ccd==49 and one ccd==50
    vis_with_49 = np.unique(visits[ccds == 49])
    vis_with_50 = np.unique(visits[ccds == 50])
    common_visits = np.intersect1d(vis_with_49, vis_with_50)
    
    # 2) build masks for those visits, split by ccd
    mask_common = np.isin(visits, common_visits)
    mask49 = mask_common & (ccds == 49)
    mask50 = mask_common & (ccds == 50)
    
    frames_49 = frames[mask49]
    frames_50 = frames[mask50]
    
    frames_49 = frames_49[np.argsort(frames_49["visit"])]
    frames_50 = frames_50[np.argsort(frames_50["visit"])]
    frames_49["ra"] = (frames_50["ra"] + frames_49["ra"]) / 2.0
    frames_49["dec"] = (frames_50["dec"] + frames_49["dec"]) / 2.0
    
    
    mask = hp.read_map("fdfc_hp_window.fits", nest=True, dtype=bool)
    theta = np.deg2rad(90.0 - frames_49["dec"])
    phi = np.deg2rad(frames_49["ra"])
    
    # Convert to HEALPix pixel indices (in NESTED ordering)
    pix = hp.ang2pix(NSIDE, theta, phi, nest=True)
    mm = mask[pix]
    frames_49 = frames_49[mm]
    frames = frames_49
    fitsio.write("/gpfs02/work/xiangchong.li/work/hsc_data/s23b/visits.fits", frames_49.to_pandas().to_records())
else:
    frames = fitsio.read("/gpfs02/work/xiangchong.li/work/hsc_data/s23b/visits.fits")

In [5]:
# Create x, y, yerr for all 8 plots
x_data = []
y_data = []
y_errs = []

def compute_xy(data_subset):
    arr = np.average(data_subset, axis=0)
    x  = arr[0]
    y1 = arr[1] / arr[3]
    y2 = arr[2] / arr[3]
    return x, y1, y2

for ff in ["cluster", "visit", "tract"]:
    data = fitsio.read(f"/gpfs02/work/xiangchong.li/work/hsc_data/s23b/deepCoadd_anacal_v2/tests/NG_{ff}.fits")

    # get the “nominal” values
    x, y1, y2 = compute_xy(data)
    
    # ---- bootstrap parameters ----
    nboot = 1000                      # number of bootstrap samples
    nreal = data.shape[0]             # how many realizations you have
    
    # pre-allocate arrays to hold each trial
    y1_boot = np.empty((nboot, y1.size))
    y2_boot = np.empty((nboot, y2.size))
    
    # ---- run the bootstrap ----
    for i in range(nboot):
        # sample `nreal` indices with replacement
        idx = np.random.choice(nreal, size=nreal, replace=True)
        # compute ratios on that resampled set
        _, y1_i, y2_i = compute_xy(data[idx])
        y1_boot[i] = y1_i
        y2_boot[i] = y2_i
    
    # ---- compute the 1-σ errors ----
    y1_err = np.std(y1_boot, axis=0)
    y2_err = np.std(y2_boot, axis=0)

    x_data.extend([x, x])
    y_data.extend([y1, y2])
    y_errs.extend([y1_err, y2_err])

In [ ]:
multiply = 100
plt.close()

_, axes = plt.subplots(1, 2, figsize=(11, 5))
ax = axes[0]
i = 2
tmin = 0
tmax = 18
chi2 = np.sum((y_data[i][tmin:tmax] / y_errs[i][tmin:tmax]) ** 2.0)
df = len(y_data[i][tmin:tmax])
pv = scipy.stats.chi2.sf(chi2, df)
ax.errorbar(
    x_data[i] * 1.04, 
    y_data[i] * x_data[i] * multiply, 
    y_errs[i] * x_data[i] * multiply,
    fmt='o', ms=4, capsize=3, linestyle='', alpha=0.5, 
    color="red",
    label=r"$\gamma_t, p =%.3f$" % (pv),
)

i = 3
chi2 = np.sum((y_data[i][tmin:tmax] / y_errs[i][tmin:tmax]) ** 2.0)
df = len(y_data[i][tmin:tmax])
pv = scipy.stats.chi2.sf(chi2, df)
ax.errorbar(
    x_data[i]/1.04, 
    y_data[i] * x_data[i] * multiply, 
    y_errs[i] * x_data[i] * multiply,
    fmt='x', ms=4, capsize=3, linestyle='', alpha=0.5, 
    color="blue", 
    label=r"$\gamma_\times, p =%.3f$" % (pv),
)
ax.set_xlim(0.25, 165)
ax.set_xscale('log')
ax.legend(fontsize=15, loc="upper left", handletextpad=0.2)
ax.axhline(0.0, ls='--', color='black')
ax.set_title("Visit", y=0.90)
ax.set_xlabel(r"$\theta\ \mathrm{[arcmin]}$")
ax.set_ylabel(r"$\langle \theta\,\gamma \rangle \times 100$")
ax.set_ylim(-0.5, 1)

ax = axes[1]
i = 4
tmin = 0
tmax = 15
chi2 = np.sum((y_data[i][tmin:tmax] / y_errs[i][tmin:tmax]) ** 2.0)
df = len(y_data[i][tmin:tmax])
pv = scipy.stats.chi2.sf(chi2, df)
ax.errorbar(
    x_data[i]* 1.04, 
    y_data[i] * x_data[i] * multiply, 
    y_errs[i] * x_data[i] * multiply,
    fmt='o', ms=4, capsize=3, linestyle='', alpha=0.5, 
    color="red",
    label=r"$\gamma_t, p=%.3f$" % (pv),
)
i = 5
chi2 = np.sum((y_data[i][tmin:tmax] / y_errs[i][tmin:tmax]) ** 2.0)
df = len(y_data[i][tmin:tmax])
pv = scipy.stats.chi2.sf(chi2, df)
ax.errorbar(
    x_data[i]/1.04, 
    y_data[i] * x_data[i] * multiply, 
    y_errs[i] * x_data[i] * multiply,
    fmt='x', ms=4, capsize=3, linestyle='', alpha=0.5,
    color="blue",
    label=r"$\gamma_\times, p=%.3f$" % (pv),
)
ax.set_xscale('log')
ax.legend(fontsize=15, loc="upper left", handletextpad=0.2)
ax.set_xlim(0.25, 54)
ax.set_ylim(-1.0, 2)
ax.axhline(0.0, ls='--', color='black')
ax.set_title("Tract", y=0.90)
ax.set_xlabel(r"$\theta\ \mathrm{[arcmin]}$")

plt.tight_layout()
plt.show()

In [ ]:
y_data[0].shape

In [ ]:
multiply = 100
plt.close()

_, axes = plt.subplots(1, 2, figsize=(11, 5))
ax = axes[0]
i = 0
chi2 = np.sum((y_data[i][0:-3] / y_errs[i][0:-3]) ** 2.0)
df = len(y_data[i][0:-3])
ax.errorbar(
    x_data[i]*1.04, 
    y_data[i] * x_data[i] * multiply, 
    y_errs[i] * x_data[i] * multiply,
    fmt='o', ms=4, capsize=3, linestyle='', alpha=0.5, 
    color="black",
    label=r"$\gamma_t$",
)

ax.set_xscale('log')
ax.legend(fontsize=15, loc="upper left", handletextpad=0.2)
ax.axhline(0.0, ls='--', color='black')
ax.set_title("Tangential", y=0.90)
ax.set_xlabel(r"$\theta\ \mathrm{[arcmin]}$")
ax.set_ylabel(r"$\langle \theta\,\gamma \rangle \times 100$")

ax = axes[1]
i = 1
chi2 = np.sum((y_data[i] / y_errs[i]) ** 2.0)
df = len(y_data[i])
pv = scipy.stats.chi2.sf(chi2, df)
ax.errorbar(
    x_data[i]/1.04, 
    y_data[i] * x_data[i] * multiply, 
    y_errs[i] * x_data[i] * multiply,
    fmt='x', ms=4, capsize=3, linestyle='', alpha=0.5,
    color="black",
    label=r"$\gamma_\times, p =%.3f$" % (pv),
)
ax.set_xscale('log')
ax.legend(fontsize=15, loc="upper left", handletextpad=0.2)
ax.set_ylim(-1, 0.5)
ax.axhline(0.0, ls='--', color='black')
ax.set_title("Cross", y=0.90)
ax.set_xlabel(r"$\theta\ \mathrm{[arcmin]}$")

plt.tight_layout()
plt.show()